## Task 3: Deploy a Model in Amazon SageMaker

In this task, you deploy the trained machine learning model using Amazon SageMaker to get predictions. You first export the model artifact to an Amazon S3 bucket. You then create a Sagemaker Model object from model artifact. Next, you create a batch transform job by specifying the Sagemaker Model, the S3 bucket location of the data to be transformed and the S3 bucket location to store the results. You then start the batch transform job and monitor its progress. Finally, once the batch transform job is complete, you retrieve the results from the specified S3 bucket. The results will contain the inferences made by the trained model on the input data.

**Note:** Solutions are available in the <a href="#appendix" target="_self">**Appendix**</a> section at the bottom of the notebook if you experience any issues completing the challenges.

### Task 3.1 Set up the environment

This basic setup code has been included to help you get started. Read and run these cells first to get packages installed and variables created.

In [1]:
#import libraries and set variable values

import boto3
import sagemaker

# Initialize the S3 client
s3_client = boto3.client('s3')

# List all S3 buckets
response = s3_client.list_buckets()

# Get the bucket name with the prefix "lab-data-bucket"
bucket_åname = None
bucket_prefix = "lab-data-bucket"
prefix = "lab3/batch-transform"

for bucket in response['Buckets']:
    if bucket['Name'].startswith(bucket_prefix):
        bucket_name = bucket['Name']
        break

if bucket_name:
    print(f"Bucket with prefix '{bucket_prefix}': {bucket_name}")
else:
    print(f"No bucket found with prefix '{bucket_prefix}'.")

bucket_path = f"s3://{bucket_name}"

# SageMaker setup
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

print(f"Bucket path: {bucket_path}")
print(f"SageMaker execution role: {role}")
print(f"Region: {region}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Bucket with prefix 'lab-data-bucket': lab-data-bucket-c6c712d0
Bucket path: s3://lab-data-bucket-c6c712d0
SageMaker execution role: arn:aws:iam::492040971417:role/sagemaker_studio_role
Region: us-east-1


### Task 3.2 Create a model

In this task, you take the pretrained model and upload it to S3. You then create a model in SageMaker. The model will later be deployed using the SageMaker Batch Transform job.

#### Requirements and Configuration

- Use **S3.Client.upload_file** API to upload the model artifact and test data to an S3 bucket.
- Upload the model artifacts to **s3://\<bucket_name\>/lab3/batch-transform/models/model.tar.gz**.
- Upload test datasets to **s3://\<bucket_name\>/test_data.csv**.


Refer to [Upload a file to an S3 object](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/upload_file.html#upload-file) for more information about using the **upload_file** method.

In [12]:
# Upload the model to the Amazon S3 bucket
from sagemaker.session import Session
from sagemaker.image_uris import retrieve

sagemaker = boto3.client('sagemaker')
s3_client = boto3.client("s3")

#add_upload-model_code_here
s3_client.upload_file(Filename="model.tar.gz", Bucket= bucket_name,  Key=f"{prefix}/models/model.tar.gz")
upload_model_path = f"s3://{bucket_name}/{prefix}/models/model.tar.gz"
print(upload_model_path)

#add_upload-dataset_code_here
s3_client.upload_file(Filename="test_data.csv", Bucket= bucket_name,  Key=f"test_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
upload_dataset_path = f"s3://{bucket_name}/test_data.csv"
print(upload_dataset_path)

s3://lab-data-bucket-c6c712d0/lab3/batch-transform/models/model.tar.gz
s3://lab-data-bucket-c6c712d0/test_data.csv


**Note:** For detailed code on how to upload the model artifact and test data to an S3 bucket, refer to <a href="#task3-2-1" target="_self">**Upload model and test data to S3 (Task 3.2)**</a> in the *Appendix* section. 

<a id="task3-2-continue"></a>

Next, create a model object in SageMaker from model artifacts above.

#### Requirements and Configuration

- Use **SageMaker.Client.create_model** API to create a model in SageMaker.
- In the Request, use the following options:
    - ModelName='telemarketing-model'
    - ExecutionRoleArn=role (The IAM role create earlier that SageMaker uses)
    - For the primary container, use the following options:
        - Image= XGBoost container image URI (retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.5-1'))
        - ModelDataUrl=model_path (The S3 path where the model artifacts are uploaded earlier) 

Refer to [Creating a model in SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_model.html) for more information about using the **SageMaker.Client.create_model** method.

In [16]:
sagemaker = boto3.client('sagemaker')

#add_create-model_code_here
image = retrieve(framework = 'xgboost', region=boto3.Session().region_name, version='1.5-1') 
primary_container = {"Image":image, "ModelDataUrl":upload_model_path}


#Set up the model
create_model_response = sagemaker.create_model(
    ModelName='telemarketing-model', ExecutionRoleArn=role, PrimaryContainer=primary_container)

print(create_model_response["ModelArn"])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6                                                                                             │
│    7                                                                                             │
│    8 #Set up the model                                                                           │
│ ❱  9 create_model_response = sagemaker.create_model(                                             │
│   10 │   ModelName='telemarketing-model', ExecutionRoleArn=role, PrimaryContainer=primary_con    │
│   11                                                                                             │
│   12 print(create_model_response["ModelArn"])                                                    │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Cannot create already 
existing model "arn:aws:sagemaker:us-east-1:492040971417:model/telemarketing-model".

**Note:** For detailed code on how to create a model, refer to <a href="#task3-2-2" target="_self">**Create a model in SageMaker (Task 3.2)**</a> in the *Appendix* section. 

<a id="task3-3"></a>

### Task 3.3 Create a batch transformer

In this task, you create a batch transformer.

Refer to [Using Transformer API](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer) for more information about using the **Transfomer** API.

#### Requirements and Configuration

- Use **sagemaker.transformer.Transformer** API to initialize a transformer in SageMaker.
- In the Parameters section, use the following options:
    - model_name="telemarketing-model"
    - instance_type="ml.m4.xlarge"
    - instance_count=1
    - assemble_with="Line"
    - strategy="MultiRecord"
    - accept="text/csv"
    - output_path=f"s3://{bucket_name}/{prefix}/batch-transform/test"

In [17]:
from sagemaker.transformer import Transformer

#add_create-batch-transformer_code_here
transformer = Transformer(
    model_name="telemarketing-model",
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path=f"s3://{bucket_name}/{prefix}/batch-transform/test"  
)

**Note:** For detailed code on how to create a Transformer object, refer to <a href="#task3-3-1" target="_self">**Create a Batch Transformer Object (Task 3.3)**</a> in the *Appendix* section. 

<a id="task3-4"></a>

### Task 3.4: Run the batch transform job

In this task, you run the transform job. You will pass the training data in to get predictions from the batch transformer.

Refer to [Using Transform API](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer.transform) for more information about using the **Transformer.transform** API.

#### Requirements and Configuration

- Use **Transformer.transform** API to initialize a transformer in SageMaker.
- In the Parameters section, use the following options:
    - batch_path (Input data location that Amazon S3 created earlier)
    - content_type="text/csv"
    - split_type="Line"
    - join_source="Input"

In [19]:
transformer.transform(
 #add_run-batch-transform-job_code_here   
    upload_dataset_path,content_type="text/csv",split_type="Line",join_source="Input"
)

transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2025-06-26-10-47-34-361


.................................../miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-06-26:10:53:19:INFO] No GPUs detected (normal if no gpus installed)
/miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-06-26:10:53:19:INFO] No GPUs detected (normal if no gpus installed)
[2025-06-26:10:53:19:INFO] No GPUs detected (normal if no gpus installed)
[2025-06-26:10:53:19:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;

In [21]:


job_name = transformer.latest_transform_job.name
response = sagemaker.describe_transform_job(TransformJobName=job_name)
print(response["TransformJobStatus"])



Completed


In [25]:
#download the output file from the specified S3 output location

s3_client.download_file(
    Bucket="lab-data-bucket-c6c712d0",
    Key="test_data.csv.out",
    Filename="predictions.csv"
)



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 #download the output file from the specified S3 output location                              │
│   2                                                                                              │
│ ❱ 3 s3_client.download_file(                                                                     │
│   4 │   Bucket="lab-data-bucket-c6c712d0",                                                       │
│   5 │   Key="test_data.csv.out",                                                                 │
│   6 │   Filename="predictions.csv"                                                               │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/boto3/s3/inject.py:192 in download_file                  │
│                                                                                                  │
│   189 │   │   transfer.                                                                          │
│   190 │   """                                                                                    │
│   191 │   with S3Transfer(self, Config) as transfer:                                             │
│ ❱ 192 │   │   return transfer.download_file(                                                     │
│   193 │   │   │   bucket=Bucket,                                                                 │
│   194 │   │   │   key=Key,                                                                       │
│   195 │   │   │   filename=Filename,                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/boto3/s3/transfer.py:406 in download_file                │
│                                                                                                  │
│   403 │   │   │   bucket, key, filename, extra_args, subscribers                                 │
│   404 │   │   )                                                                                  │
│   405 │   │   try:                                                                               │
│ ❱ 406 │   │   │   future.result()                                                                │
│   407 │   │   # This is for backwards compatibility where when retries are                       │
│   408 │   │   # exceeded we need to throw the same error from boto3 instead of                   │
│   409 │   │   # s3transfer's built in RetriesExceededError as current users are                  │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/s3transfer/futures.py:103 in result                      │
│                                                                                                  │
│   100 │   │   │   # Usually the result() method blocks until the transfer is done,               │
│   101 │   │   │   # however if a KeyboardInterrupt is raised we want want to exit                │
│   102 │   │   │   # out of this and propagate the exception.                                     │
│ ❱ 103 │   │   │   return self._coordinator.result()                                              │
│   104 │   │   except KeyboardInterrupt as e:                                                     │
│   105 │   │   │   self.cancel()                                                                  │
│   106 │   │   │   raise e                                                                        │
│                                                            

**Note:** For detailed code on how to run a transform job, refer to <a href="#task3-4-1" target="_self">**Run a batch transform job (Task 3.4)**</a> in the *Appendix* section. 

<a id="task3-5"></a>

### Task 3.5: Verify the output stored in an S3 bucket

In this task, you pull the the output of the Batch Transform job in S3. It should show the list of predictions.

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
!head test_data.csv.out

Congratulations! you have successfully deployed the trained machine learning model using Amazon SageMaker to get predictions.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with the **Conclusion**.

## Additional resources

- [Deploy models for inference](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)
- [Using SageMaker APIs](https://sagemaker.readthedocs.io/en/stable/api/inference/index.html)

<a name="appendix" id="appendix"></a>

## Appendix


<a name="task3-2-1" id="task3-2-1"></a>

### Upload model and test data to S3 (Task 3.2)

In [ ]:
# Upload the model to the Amazon S3 bucket
from sagemaker.session import Session
from sagemaker.image_uris import retrieve

#session = boto3.Session()
sagemaker = boto3.client('sagemaker')
s3_client = boto3.client("s3")

#upload-model
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket_name, Key=f"{prefix}/models/model.tar.gz")  
model_path = f"s3://{bucket_name}/{prefix}/models/model.tar.gz"
print(model_path)

#upload-dataset
s3_client.upload_file(Filename="test_data.csv", Bucket=bucket_name, Key=f"test_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
batch_path = f"s3://{bucket_name}/test_data.csv"
print(batch_path)

To continue this lab, return to <a href="#task3-2-continue" target="_self">Task 3.2</a>.

<a name="task3-2-2" id="task3-2-2"></a>

### Create a model in SageMaker (Task 3.2)

In [ ]:
sagemaker = boto3.client('sagemaker')

# Retrieve the container image
image = retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.5-1')

primary_container = {"Image": image, "ModelDataUrl": model_path}

# Set up the model
create_model_response = sagemaker.create_model(
    ModelName='telemarketing-model', ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

To continue this lab, return to <a href="#task3-3" target="_self">Task 3.3</a>.

<a name="task3-3-1" id="task3-3-1"></a>

### Create a Batch Transformer Object (Task 3.3)

In [ ]:
from sagemaker.transformer import Transformer

#create-batch-transformer
transformer = Transformer(
    model_name="telemarketing-model",
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path=f"s3://{bucket_name}/{prefix}/batch-transform/test"
)

To continue this lab, return to <a href="#task3-4" target="_self">Task 3.4</a>.

<a name="task3-4-1" id="task3-4-1"></a>

### Run a batch transform job (Task 3.4)


In [ ]:
#run-batch-transform-job
transformer.transform(batch_path, content_type="text/csv", split_type="Line", join_source="Input")

transformer.wait()

To continue this lab, return to <a href="#task3-5" target="_self">Task 3.5</a>.